# Partie 3. - Extraction d'informations

Dans cette partie, nous allons mettre en oeuvre des techniques d'extraction d'informations.

A partir du texte extrait des documents, l'objectif est d'extraire des informations métiers intéressantes comme des références, des noms de personnes, des lieux géographiques etc. 

L'extraction d'informations peut être très utile lorsque l'on cherche à _structurer_ des données qui ne le sont pas de base, comme du texte brut. Les cas d'usage les plus fréquents sont :
- trouver et comprendre des portions de textes pertinentes pour la réalisation d'une tâche;
- calculer des statistiques sur les objets les plus fréquents, par exemple les personnes les plus cités dans des articles;
- trouver des liens entre des objets, par exemple les liens entre des sociétés.

## Import des bibliothèques logicielles et configuration

In [12]:
import re # mise en oeuvre d'expression régulières
import ast # à ignorer pour ce TP
import spacy
from spacy import displacy
!python -m spacy download en_core_web_sm
!python -m spacy download fr_core_news_sm
!python -m spacy download fr_core_news_md
!python -m spacy download fr_dep_news_trf


You should consider upgrading via the '/home/vincent/dev/FORMATION_DATA_SCIENCES/taln-course/venv/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
You should consider upgrading via the '/home/vincent/dev/FORMATION_DATA_SCIENCES/taln-course/venv/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
You should consider upgrading via the '/home/vincent/dev/FORMATION_DATA_SCIENCES/taln-course/venv/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_md')
You should consider upgrading via the '/home/vincent/dev/FORMATION_DATA_SCIENCES/taln-course/venv/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('fr_dep_news_trf')


In [4]:
# Configuration

## 1. Extraction d'informations par expressions régulières

Une **expression régulière** est chaîne de caractères décrivant un ensemble de chaînes de caractères possibles.

Tout d'abord, chargement le jeu de données créé en Partie 1. Nous travaillerons sur un sous ensemble de ce jeu de données pour garantir des temps de réponse acceptables.

In [5]:
f = open("./dataset_processed/dataset.json", "r")
documents_raw = f.read()
documents = ast.literal_eval(documents_raw)
f.close()
big_text = ' '.join([doc['content'] for doc in documents.values()])
big_text = big_text[0:1000000] # sélection d'un sous ensemble

### Extraction des lois et décrets

Les lois et décrets ont un numéro codifié. Ce dernier se compose de l'année sur 2 ou 4 chiffres, suivi d'un tiret (-), suivi d'un numéro sur au moins 2 chiffres. Par exemple : 
- loi n° 2013-595
- décret n° 2011-1503

**Exercice** : écrire l'expression régulière permettant d'extraire l'ensemble des lois et des décrets cités dans le jeu de données. Afficher le résultat.

In [6]:
url_regex = r" ([0-9]{4}-[0-9]{2,4}) " # TODO
match = set(re.findall(url_regex, big_text, re.IGNORECASE))
print(match)


{'2011-267', '2007-1434', '2010-1250', '2122-18', '2013-1113', '2016-847', '2122-22', '2013-745', '2015-1380', '2007-1787', '3132-27', '2311-13', '2003-239', '2001-1062', '2010-569', '2004-193', '2007-297', '2122-17', '1331-22', '2223-27', '2015-990', '2020-2022', '2121-30', '2004-192', '2007-1720', '2019-2020', '2000-277'}


### Extraction des mails

Un autre exemple est celui de l'extraction des adresses email dans des textes. Une adresse email se décomposer de la manière suivante : `texte@nom_hote.domaine`.

**Exercice** : écrire l'exression régulière permettant d'extraire l'ensemble des adresses email citées dans le jeu de données. Afficher le résultat.

In [7]:
mail_regex = '[\w\.-]+@[\w\.-]+' # TODO expression régulière permettant de cibler un mail.
match = re.findall(mail_regex, big_text)
for i in match:
    print(i)


bulot@sdis95.fr
mae.kurkjian@one.org
qparrinello@oxfamfrance.org
contact@asso-sherpa.org
contact@lebasic.com
contact@pirae.pf
ademe.polynesie@ademe.fr
francis@fenuama.pf
g.tekohuotetua@pirae.pf
contact@atmo-france.org
contact@atmo-france.org


### Extraction d'URL

L'extraction d'URL par expression régulière n'est si triviale qu'elle n'y paraît. Aujourd'hui, les navigateurs acceptent des URLs même si elles ne sont pas parfaitement formées. Par exemple, il n'est pas obligatoire de spécifier `https` ou `www`. 

Voici ci-dessous un exemple d'expression régulière permettant d'extraire les URLs, et elle n'est pas simple à comprendre. Essayons là en lançant le bloc de codes ci-dessous.

In [9]:
url_regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
match = set(re.findall(url_regex, big_text))
for i in match:
    print(i[0])

http://www.gfintegrity
http://srv-bustracker/
http://china.areva.com/home/
http://www.opengeospatial.org/
www.oxfamfrance.org
un.org/db/
publishwhatyoupay.org/our-work/
http://www.sonangol.co.ao/
http://graphics.thomsonreuters
http://www.afrique7.com/
investingnews.com/daily/resource-
http://www.citepa.org/fr/activites/inventaires-
http://www.engie
https://www.imf.org/external/pubs/
http://www.minfin.gov
http://www.total.com/site
http://geopython.github.io/OWSLib/
http://www.stat-niger.org/
http://94.143.218.36/
http://www.lemonde.fr/afrique/
https://www.publiceye.ch/fr/
http://www.oecd-ilibrary.org/
http://www.maureletprom.fr/
https://www.oxfam.org/sites/
http://cnig.gouv.fr/wp-content/
http://www
atmo-aauvergnerhonealpes.fr/donnees-ouvertes-de-qualite-de-lair
http://unctadstat
http://www.jeuneafrique
https://www.emis.com/sites/
http://www.itieniger.ne/images/
http://investingnews.com/daily/
https://www.eia.gov/beta/
https://mining-atlas.com/
www.oxfam.org/files/niger_
http://www.eram

## 2. Extraction d'informations via un modèle entraîné.

Les expressions régulières sont un puissant outil pour extraire de l'information structurée mais uniquement lorsque l'on est capable de décrire précisément et sans ambiguïté ce que l'on cherche.

Dans certains cas, cela est impossible ou très difficile. Par exemple lorsque l'on veut extraire des noms de personnes, des lieux géographiques ou encore des noms de société. Dans ces cas là, nous ne pouvons plus utiliser les expressions régulières car la variabilité des cas possibles est trop grande.

Pour répondre à ce problème, nous pouvons utiliser des modèles statistiques ayant préalablement été entraînés à identifier dans un texte les séquences de mots reflétant ce que l'on souhaite extraire.

Un exemple bien connu de ce type d'extraction est la **Reconnaissance d'Entités Nommées** (_Named Entity Recognition_). Une entité nommée est un mot ou une séquence de mots catégorisable dans des classes. Les classes les plus connues sont : `Personne`, `Organisation`, `Lieu géographique`, `Montant`, `Quantité`, `Date`, `Distance`.

Dans le code ci-dessous, nous allons mettre en oeuvre l'extraction d'entités nommées.

Premièrement, chargeons avec la bibliothèque `spacy` un modèle ayant préalablement été entraîné à reconnaître les entités nommées dans des textes en langue française.

In [15]:
nlp = spacy.load('fr_core_news_md') # Chargement d'un modèle de TALN pour le français.

Avant d'exploiter le jeu de données créé précédemment, nous allons travailler sur le texte de taille réduite ci-dessous (extrait de Wikipedia) afin de mieux visualiser les extractions, leurs forces et leurs limites.

In [17]:
text = '''
La société Naval Group est un groupe industriel français spécialisé dans la construction navale de défense. 
Le groupe emploie 15 792 personnes en 2020 à travers dix-huit pays. 
Société de droit privé détenue principalement à hauteur de 62,49 % par l’État français et de 35 % par Thales, 
Naval Group est, depuis 2017, l’héritière des arsenaux français et de la Direction des constructions et armes navales (DCAN), 
devenue la Direction des constructions navales (DCN) en 1991 et DCNS en 2007 (le « s » ajouté pour la notion de système et de service). 
Depuis 2021, le groupe se recentre sur ses activités navales. 
'''

Lançons maintenant l'analyse du texte. La ligne `nlp(text)` réalise de nombreuses opérations comme la _tokenisation_ et la normalisation du texte en plus de l'extraction des entités nommées.

In [22]:
doc = nlp(text)

# Affichage des entités nommées.
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Naval Group 12 23 ORG
Société de droit 179 195 ORG
État 252 256 LOC
Thales 281 287 LOC
Direction des constructions et armes navales 363 407 ORG
DCAN 409 413 ORG
Direction des constructions navales 428 463 ORG
DCN 465 468 ORG
DCNS 481 485 ORG


Nous pouvons aussi afficher les résultats de manière plus visuelle.

In [23]:
displacy.render(doc, style="ent")


: 

Les résultats permettent d'identifier des informations itnéressantes, comme des organisations, que celles-ci soient écrites sous la forme d'un acronyme ou pas. Le modèle n'est néanmoins pas parfait et des erreurs peuvent survenir. _Thales_ est par exemple reconnu comme une localisation géographique. _Naval Group_ à la troisième ligne n'est pas reconnu comme une organisation.

Ces résultats pourraient être améliorés en utilisant un modèle plus conséquent. Pour la bibliothèque _Spacy_ que nous utilisons, la liste des modèles pour la langue française est disponible ici https://spacy.io/models/fr#fr_dep_news_trf. 